# Synthetic Example

This notebook includes independent scripts for training a neural bias potential with GFlownets and evaluating the realisticity and diversity of our model on a synthetic system.

### Imports

In [ ]:
import torch
import numpy as np
from torch import nn
from tqdm import tqdm
import matplotlib.pyplot as plt

### System Configuration


In [ ]:
seed = 2
device = "cuda" if torch.cuda.is_available() else "cpu"
save_dir = "results"

### Model Configuration

In [ ]:
force = False
dist_feat = True

### Sampling Configuration

In [ ]:
sigma = 1
num_steps = 1000
timestep = 0.01
num_samples = 512
temperature = 1200
kB = 8.6173303e-5 
kbT = kB * temperature
std = np.sqrt(2 * kbT * timestep)
normal = torch.distributions.Normal(0, std)
start_position = torch.tensor([-1.118, 0], dtype=torch.float32).to(device)
target_position = torch.tensor([1.118, 0], dtype=torch.float32).to(device)

### Training Configuration


In [ ]:
log_z_lr = 0.01
policy_lr = 0.001
batch_size = 1024
num_rollouts = 1000
buffer_size = 50000
trains_per_rollout = 100
start_temperature = 4800
# start_temperature = 1200
end_temperature = 1200
kbTs = (
    torch.linspace(start_temperature, end_temperature, num_rollouts) * kB
)
train_stds = torch.sqrt(2 * kbTs * timestep)

### Synthetic System

In [ ]:
def system(pos):
    pos.requires_grad_(True)
    x = pos[:, 0]
    y = pos[:, 1]
    term_1 = 4 * (1 - x**2 - y**2) ** 2
    term_2 = 2 * (x**2 - 2) ** 2
    term_3 = ((x + y) ** 2 - 1) ** 2
    term_4 = ((x - y) ** 2 - 1) ** 2
    potential = (term_1 + term_2 + term_3 + term_4 - 2.0) / 6.0
    force = -torch.autograd.grad(potential.sum(), pos)[0]
    return potential, force

### Neural Bias Potential

In [ ]:
class NeuralBias(nn.Module):
    def __init__(self):
        super().__init__()

        if force:
            self.output_dim = 2
        else:
            self.output_dim = 1

        if dist_feat:
            self.input_dim = 3
        else:  
            self.input_dim = 2

        self.mlp = nn.Sequential(
            nn.Linear(self.input_dim, 8),
            nn.ReLU(),
            nn.Linear(8, 4),
            nn.ReLU(),
            nn.Linear(4, self.output_dim, bias=False),
        )

        self.log_z = nn.Parameter(torch.tensor(0.0))

        self.to(device)

    def forward(self, pos):
        if not force:
            pos.requires_grad = True
        if dist_feat:
            dist = torch.norm(pos - target_position, dim=-1, keepdim=True)
            pos_ = torch.cat([pos, dist], dim=-1)
        else:
            pos_ = pos

        out = self.mlp(pos_.reshape(-1, self.input_dim))

        if not force:
            f = -torch.autograd.grad(
                out.sum(), pos, create_graph=True, retain_graph=True
            )[0]
        else:
            f = out.view(*pos.shape)

        return f

### TPS-GFN

In [ ]:
class FlowNetAgent:
    def __init__(self):
        self.policy = NeuralBias()
        self.replay = ReplayBuffer()

    def sample(self, std, training=True):
        positions = torch.zeros(
            (num_samples, num_steps + 1, 2),
            device=device,
        )
        actions = torch.zeros(
            (num_samples, num_steps, 2),
            device=device,
        )
        noises = torch.normal(
            torch.zeros(
                (num_samples, num_steps, 2),
                device=device,
            ),
            torch.ones(
                (num_samples, num_steps, 2),
                device=device,
            ),
        )
        potentials = torch.zeros((num_samples, num_steps + 1), device=device)

        potential = system(start_position.unsqueeze(0))[0]

        position = start_position.unsqueeze(0)
        positions[:, 0] = position
        potentials[:, 0] = potential


        for s in range(num_steps):
            noise = noises[:, s]
            bias = self.policy(position.detach()).squeeze().detach()
            # bias = torch.zeros_like(noise)
            potential, force = system(position)
            mean = position + force * timestep
            position = position + (force + bias) * timestep + std * noise
            positions[:, s + 1] = position
            potentials[:, s + 1] = potential
            actions[:, s] = position - mean

        log_md_reward = normal.log_prob(actions.detach())
        log_target_reward = (
            -0.5
            * torch.square((positions - target_position.view(1, 1, -1)) / std).mean(2)
            / sigma
        )
        log_target_reward, last_idx = log_target_reward.max(1)
        log_reward = log_md_reward.mean((1, 2)) + log_target_reward

        if training:
            self.replay.add((positions.detach(), actions.detach(), log_reward.detach()))

        log = {
            "positions": positions,
            "potentials": potentials,
            "log_likelihood": log_md_reward.sum(-1).mean(1),
            "last_idx": last_idx,
        }
        return log

    def train(self):
        optimizer = torch.optim.Adam(
            [
                {"params": [self.policy.log_z], "lr": log_z_lr},
                {"params": self.policy.mlp.parameters(), "lr": policy_lr},
            ]
        )

        positions, actions, log_reward = self.replay.sample()

        biases = self.policy(positions[:, :-1].detach())

        log_z = self.policy.log_z
        log_forward = normal.log_prob(actions-biases).mean((1, 2))

        loss = (log_z + log_forward - log_reward).square().mean()

        loss.backward()

        optimizer.step()
        optimizer.zero_grad()
        return loss.item()
    
    
class ReplayBuffer:
    def __init__(self):
        self.positions = torch.zeros(
            (buffer_size, num_steps + 1, 2),
            device=device,
        )
        self.actions = torch.zeros(
            (buffer_size, num_steps, 2), device=device
        )
        self.log_reward = torch.zeros(buffer_size, device=device)

        self.idx = 0
        self.buffer_size = buffer_size
        self.num_samples = num_samples

    def add(self, data):
        indices = torch.arange(self.idx, self.idx + self.num_samples) % self.buffer_size
        self.idx += self.num_samples

        self.positions[indices], self.actions[indices], self.log_reward[indices] = data

    def sample(self):
        indices = torch.randperm(min(self.idx, self.buffer_size))[: self.num_samples]
        return self.positions[indices], self.actions[indices], self.log_reward[indices]

### Metrics and Plot

In [ ]:
class Logger:
    def __init__(self):
        self.plot_samples = 64

    def log(self, positions, potentials, log_likelihood, last_idx):
        dist, dist_std = self.expected_distance(positions, last_idx)
        ll, ll_std = self.log_likelihood(log_likelihood)
        thp, etp, efp, etp_std, efp_std = self.cv_metrics(positions, potentials, last_idx)
        if thp > 0:
            print(f"Rollout: {rollout}, ED: {dist:.2f} (±{dist_std:.2f}), THP: {thp:.2f}, ETP: {etp:.2f} (±{etp_std:.2f}), LL: {ll:.2f} (±{ll_std:.2f}), EFP: {efp:.2f} (±{efp_std:.2f})")
        else:
            print(f"Rollout: {rollout}, ED: {dist:.2f} (±{dist_std:.2f}), THP: None, ETP: None, LL: {ll:.2f} (±{ll_std:.2f}), EFP: None")

    def expected_distance(self, positions, last_idx):
        last_position = positions[torch.arange(len(positions)), last_idx]
        dists = (last_position - target_position.unsqueeze(0)).square().mean((1))
        return dists.mean().item(), dists.std().item()

    def cv_metrics(self, positions, potentials, last_idx):
        etps, efps, etp_idxs, efp_idxs = [], [], [], []
        last_position = positions[torch.arange(len(positions)), last_idx]
        hits = (last_position - target_position.unsqueeze(0)).square().sum(1).sqrt() < 0.5

        for i, hit_idx in enumerate(hits):
            if hit_idx:
                etp, idx = potentials[i][: last_idx[i] + 1].max(0)
                etps.append(etp)
                etp_idxs.append(idx.item())

                efp = potentials[i][last_idx[i]]
                efps.append(efp)
                efp_idxs.append(last_idx[i].item())

        if len(etps) > 0:
            etps = torch.tensor(etps)
            efps = torch.tensor(efps)

            etp = etps.mean().item()
            efp = efps.mean().item()

            etp_std = etps.std().item()
            efp_std = efps.std().item()
        else:
            etp = None
            efp = None

            etp_std = None
            efp_std = None
            
        thp = 100 * hits.sum() / len(hits)
        return thp, etp, efp, etp_std, efp_std

    def log_likelihood(self, log_likelihood):
        return log_likelihood.mean().item(), log_likelihood.std().item()

    def plot_paths(self, save_dir, rollout, positions, last_idx):
        fig, ax = plt.subplots(figsize=(7, 7))
        positions = positions[:self.plot_samples].detach().cpu().numpy()
    
        z_num = 100
        circle_size = 1200
        saddle_size = 2400

        plt.xlim(-1.5, 1.5)
        plt.ylim(-1.5, 1.5)
        x = np.linspace(-1.5, 1.5, 400)
        y = np.linspace(-1.5, 1.5, 400)
        X, Y = np.meshgrid(x, y)
        
        term_1 = 4 * (1 - X**2 - Y**2) ** 2
        term_2 = 2 * (X**2 - 2) ** 2
        term_3 = ((X + Y) ** 2 - 1) ** 2
        term_4 = ((X - Y) ** 2 - 1) ** 2
        Z = (term_1 + term_2 + term_3 + term_4 - 2.0) / 6.0
    
        ax.contourf(X, Y, Z, levels=z_num, zorder=0, vmax=3)
    
        # Plot start and end positions
        ax.scatter([start_position[0].item()], [start_position[1].item()], 
                   edgecolors='black', c='w', zorder=z_num, s=circle_size)
        ax.scatter([target_position[0].item()], [target_position[1].item()], 
                   edgecolors='black', c='w', zorder=z_num, s=circle_size)
    
        # Plot saddle points (you may need to adjust these coordinates)
        saddle_points = [(0, 1), (0, -1)]
        for saddle in saddle_points:
            ax.scatter(saddle[0], saddle[1], edgecolors='black', c='w', 
                       zorder=z_num, s=saddle_size, marker="*")
    
        cm = plt.get_cmap('gist_rainbow')
        
        ax.set_prop_cycle(color=[cm(1. * i / len(positions)) for i in range(len(positions))])
        
        for i in range(len(positions)):
            ax.plot(positions[i, :last_idx[i], 0], positions[i, :last_idx[i], 1], 
                    marker='o', linestyle='None', markersize=2, alpha=1., zorder=z_num-1)
    
        # Plot basic configs
        ax.set_xlabel('x', fontsize=24, fontweight='medium')
        ax.set_ylabel('y', fontsize=24, fontweight='medium')
        ax.tick_params(left=False, right=False, labelleft=False, 
                       labelbottom=False, bottom=False)
        plt.tight_layout()
    
        plt.savefig(f"{rollout}.png")
        plt.show()
        plt.close()
        
        return fig

### Training

In [ ]:
torch.manual_seed(seed)
agent = FlowNetAgent()
logger = Logger()

for rollout in range(num_rollouts):
    num_samples = 512
    log = agent.sample(train_stds[rollout])
    logger.log(**log)

    for _ in range(trains_per_rollout):
        agent.train()
    
    if rollout%200 == 0 or rollout==num_rollouts-1:
        logger.plot_paths(save_dir, rollout, log["positions"], log["last_idx"])